<a href="https://colab.research.google.com/github/utkarshbelkhede/Financial_Dashboard/blob/master/notebooks/6_Fine_Tuning_Text_Classification_SEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets -q

     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 451 kB 77.2 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 
     |████████████████████████████████| 7.6 MB 51.1 MB/s 
     |████████████████████████████████| 132 kB 62.2 MB/s 
     |████████████████████████████████| 212 kB 80.5 MB/s 
     |████████████████████████████████| 127 kB 63.8 MB/s 


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import numpy as np
import pandas as pd

import torch

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Documents/Market Intelligence/datasets/final_labeled_sentement_analysis.csv")
df.head()

,sentence,label
0,"The persistence of COVID-19, and the preventa...",NEGATIVE
1,Item 1B. Unresolved Staff Comments None. Ite...,NEGATIVE
2,Our corporate headquarters are located in Men...,POSITIVE
3,Meta Platforms Ireland is subject to litigati...,NEGATIVE
4,"Meta Platforms, Inc. has never declared or pa...",NEGATIVE


In [ ]:
df.label.unique()

array(['NEGATIVE', 'POSITIVE'], dtype=object)

In [ ]:
target_map = {
    'POSITIVE':1,
    'NEGATIVE':0
}

df['label'] = df['label'].map(target_map)

In [ ]:
df.to_csv('data.csv', index=None)

In [ ]:
!head data.csv

sentence,label
" The persistence of COVID-19, and the preventative measures implemented to help limit the spread of the illness, have impacted, and will continue to impact, our ability to operate our business . We receive formal and informal inquiries from government authorities and regulators regarding our compliance with laws and regulations . The European Commission has conducted investigations in multiple countries focusing on whether local country tax rulings or tax legislation provides preferential tax treatment that violates European Union state aid rules .",0
" Item 1B.  Unresolved Staff Comments None. Item 1A. Unresolve Staff Comments . Item 1b: ""Unresolved staff Comments"" Item 1C: ""We are happy to have a solution to this problem."" Item 1D: ""I am not satisfied with this issue. I am happy with this resolution""",0
" Our corporate headquarters are located in Menlo Park, California . As of December 31, 2021, we owned and leased approximately 10 million square feet of office 

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset('csv', data_files='data.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3d07c8ff77f09b3a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 441
    })
})

In [ ]:
split = raw_dataset['train'].train_test_split(test_size=0.3, seed=42)
split

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 308
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 133
    })
})

In [ ]:
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
def tokenizer_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

In [ ]:
tokenized_datasets = split.map(tokenizer_fn, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoConfig

In [ ]:
config = AutoConfig.from_pretrained(checkpoint)

In [ ]:
config

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

In [ ]:
#config.id2label

In [ ]:
#config.id2label = {0: 'NEGITIVE', 1: 'POSITIVE'}

In [ ]:
#config.label2id

In [ ]:
#config.label2id = {'NEGITIVE': 0, 'POSITIVE': 1}

In [ ]:
#config

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
!pip install torchinfo -q

In [ ]:
from torchinfo import summary

In [ ]:
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

In [ ]:
training_args = TrainingArguments(
    checkpoint,
    evaluation_strategy='epoch',
    num_train_epochs=3,
    learning_rate=1e-5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    save_total_limit = 3,
    fp16 = False,
    push_to_hub = True,
)

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  prediction = np.argmax(logits, axis=-1)
  acc = np.mean(prediction == labels)
  f1 = f1_score(labels, prediction, average='macro')

  return {'accuracy': acc, 'f1': f1}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/utkarshbelkhede/distilbert-base-uncased-finetuned-sst-2-english into local empty directory.


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 308
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is fas

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.238983,0.917293,0.787138


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=20, training_loss=0.32157201766967775, metrics={'train_runtime': 232.1072, 'train_samples_per_second': 1.327, 'train_steps_per_second': 0.086, 'total_flos': 8977770963600.0, 'train_loss': 0.32157201766967775, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-sst-2-english
Configuration saved in distilbert-base-uncased-finetuned-sst-2-english/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst-2-english/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst-2-english/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst-2-english/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.30k/255M [00:00<?, ?B/s]

Upload file runs/Dec06_18-30-32_88d7da459319/events.out.tfevents.1670351446.88d7da459319.73.0:  73%|#######3  …

Upload file training_args.bin:  98%|#########8| 3.30k/3.36k [00:00<?, ?B/s]

Upload file runs/Dec06_18-30-32_88d7da459319/1670351446.859293/events.out.tfevents.1670351446.88d7da459319.73.…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/utkarshbelkhede/distilbert-base-uncased-finetuned-sst-2-english
   5e4b753..d6c57be  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/utkarshbelkhede/distilbert-base-uncased-finetuned-sst-2-english
   5e4b753..d6c57be  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}}
To https://huggingface.co/utkarshbelkhede/distilbert-base-uncased-finetuned-sst-2-english
   d6c57be..0967274  main -> main

   d6c57be..0967274  main -> main



'https://huggingface.co/utkarshbelkhede/distilbert-base-uncased-finetuned-sst-2-english/commit/d6c57be27ddaf104ca2abf1d0eda2c62366b4548'